In [1]:
from kinetics_modelling.config import PROCESSED_DATA_DIR
import polars as pl
import altair as alt
alt.data_transformers.enable("vegafusion")



2025-04-01 15:47:11.630 | INFO     | kinetics_modelling.config:<module>:12 - PROJ_ROOT path is: /faststorage/project/mutationalscanning/Workspaces/chcharlton/kinetics_modelling


DataTransformerRegistry.enable('vegafusion')

In [2]:
q = (
    pl.scan_parquet(PROCESSED_DATA_DIR / 'ob006-run1')
    .head()
)
q.collect()

contig,read_name,read_is_reverse,fn,rn,ref_center,read_center,center_seq,center_qual,center_ipd_fwd_0,center_ipd_fwd_1,center_ipd_fwd_2,center_ipd_rev_0,center_ipd_rev_1,center_ipd_rev_2,center_pw_fwd_0,center_pw_fwd_1,center_pw_fwd_2,center_pw_rev_0,center_pw_rev_1,center_pw_rev_2,center_sx_0,center_sx_1,center_sx_2,center_sm_0,center_sm_1,center_sm_2,window_sm,window_sx,window_seq,window_qual,window_ipd_fwd,window_ipd_rev,window_pw_fwd,window_pw_rev
str,str,bool,i64,i64,i64,i64,str,list[u8],u16,u16,u16,u16,u16,u16,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,list[u16],list[u16],list[str],list[u8],list[u16],list[u16],list[u8],list[u8]
"""h1tg000002l""","""m84108_240530_140750_s2/232522…",true,6,7,3846,160,"""CAA""","[93, 93, 93]",19,18,5,10,10,11,14,10,9,19,13,6,0,0,0,13,13,14,"[13, 13, … 10]","[1, 0, … 1]","[""A"", ""A"", … ""C""]","[93, 93, … 93]","[16, 8, … 40]","[12, 12, … 19]","[12, 9, … 7]","[14, 20, … 9]"
"""h1tg000002l""","""m84108_240530_140750_s2/232522…",true,6,7,3910,224,"""ATA""","[93, 93, 93]",25,18,16,68,38,20,18,8,6,5,10,8,0,0,2,13,13,10,"[13, 11, … 12]","[0, 2, … 2]","[""A"", ""T"", … ""A""]","[93, 93, … 93]","[18, 29, … 9]","[32, 12, … 17]","[8, 10, … 16]","[20, 15, … 12]"
"""h1tg000002l""","""m84108_240530_140750_s2/232522…",true,6,7,3974,288,"""AAG""","[93, 93, 93]",10,26,11,46,6,22,14,17,18,12,11,7,0,0,3,14,14,11,"[13, 14, … 14]","[1, 0, … 0]","[""A"", ""T"", … ""T""]","[93, 93, … 93]","[21, 8, … 17]","[50, 31, … 10]","[10, 14, … 13]","[7, 13, … 9]"
"""h1tg000002l""","""m84108_240530_140750_s2/232522…",true,6,7,4039,352,"""TAC""","[93, 93, 93]",22,11,19,21,22,46,15,11,13,10,13,10,2,2,0,12,11,13,"[13, 13, … 14]","[1, 0, … 0]","[""T"", ""A"", … ""A""]","[93, 93, … 93]","[10, 16, … 9]","[4, 26, … 41]","[17, 16, … 12]","[15, 27, … 5]"
"""h1tg000002l""","""m84108_240530_140750_s2/232522…",true,6,7,4103,416,"""TAC""","[93, 93, 93]",15,8,15,11,12,9,11,12,8,16,14,17,1,0,0,9,14,14,"[12, 13, … 14]","[1, 0, … 0]","[""T"", ""A"", … ""G""]","[93, 93, … 93]","[11, 25, … 20]","[19, 25, … 8]","[12, 8, … 14]","[6, 5, … 15]"


In [3]:

q = (
    pl.scan_parquet(PROCESSED_DATA_DIR / 'ob006-run1')
    .select(
        'center_qual', 
        'center_seq', 
        'fn', 
        'center_sx_0',
        'center_sx_1',
        'center_sx_2',
        'center_sm_0',
        'center_sm_1',
        'center_sm_2',
        'center_ipd_fwd_0', 
        'center_ipd_rev_0', 
        'center_pw_fwd_0',
        'center_pw_rev_0',
        'center_ipd_fwd_1', 
        'center_ipd_rev_1', 
        'center_pw_fwd_1',
        'center_pw_rev_1',
        'center_ipd_fwd_2', 
        'center_ipd_rev_2', 
        'center_pw_fwd_2',
        'center_pw_rev_2',
    )
    .with_columns(
        (pl.col('center_qual').list[0].alias('center_qual_0')),
        (pl.col('center_qual').list[1].alias('center_qual_1')),
        (pl.col('center_qual').list[2].alias('center_qual_2'))
    )
    .drop('center_qual')
    .group_by('center_qual_2')
    .agg(pl.all().head(1000))
    .explode(pl.exclude('center_qual_2'))
    )

df = q.collect()

In [6]:
df.write_parquet('../data/processed/ob006-run1-sample.parquet')


In [5]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

X = df.select(
    'fn', 
    'center_sx_0',
    'center_sx_1',
    'center_sx_2',
    'center_sm_0',
    'center_sm_1',
    'center_sm_2',
    'center_ipd_fwd_0', 
    'center_ipd_rev_0', 
    'center_pw_fwd_0',
    'center_pw_rev_0',
    'center_ipd_fwd_1', 
    'center_ipd_rev_1', 
    'center_pw_fwd_1',
    'center_pw_rev_1',
    'center_ipd_fwd_2', 
    'center_ipd_rev_2', 
    'center_pw_fwd_2',
    'center_pw_rev_2')
y = df.select('center_qual_2')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [ ]:
# reg = linear_model.Lasso(alpha=.5)
# reg.fit(X_train, y_train)